In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
import cupy as cp
from sklearnex import patch_sklearn

In [2]:
def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    return data

def mainBareBones():
    # Suppress warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with your dataset's path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes to reduce memory usage
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical variables to numeric with one-hot encoding
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = data.drop(columns=['Risk_Flag', 'Id'])  # Dropping Id and target column
    y = data['Risk_Flag']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Convert data to optimized types
    print("\nConverting data to optimized types...")
    X = X.astype(np.float32)
    y = y.astype(np.int32)

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using MLPClassifier
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

if __name__ == "__main__":
    mainBareBones()

Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Converting data to optimized types...

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the MLP model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     44147
           1       0.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.44      0.50      0.47     50400
weighted 

In [3]:
def optimize_datatypes(data):
    # Downcast numerical columns
    for col in data.select_dtypes(include=['int']):
        data[col] = cp.array(data[col], dtype=cp.int32).get()
    for col in data.select_dtypes(include=['float']):
        data[col] = cp.array(data[col], dtype=cp.float32).get()
    return data

def mainCuPY():
    # Suppress warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with your dataset's path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical variables to numeric with one-hot encoding
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = cp.array(data.drop(columns=['Risk_Flag', 'Id']).values, dtype=cp.float32).get()  # Convert to CuPy
    y = cp.array(data['Risk_Flag'].values, dtype=cp.int32).get()
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using MLPClassifier
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

if __name__ == "__main__":
    mainCuPY()


Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the MLP model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     44147
           1       0.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.44      0.50      0.47     50400
weighted avg       0.77      0.88      0.82     

In [4]:
patch_sklearn() # Apply Intel optimizations

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    return data

def mainIntelex():
    # Suppress warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with your dataset's path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes to reduce memory usage
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical variables to numeric with one-hot encoding
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = data.drop(columns=['Risk_Flag', 'Id'])  # Dropping Id and target column
    y = data['Risk_Flag']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Convert data to optimized types
    print("\nConverting data to optimized types...")
    X = X.astype(np.float32)
    y = y.astype(np.int32)

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using MLPClassifier
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

if __name__ == "__main__":
    mainIntelex()


Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Converting data to optimized types...

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the MLP model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     44147
           1       0.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.44      0.50      0.47     50400
weighted 